# Exercise 2 - Interactive charting with plotly

## 2. Price on memory

a) Recreate this visualization using plotly, make it as close as possible to the image.

In [50]:
import pandas as pd

df_price = pd.read_csv("data/historical-cost-of-computer-memory-and-storage.csv")
df_price.head()

,Entity,Code,Year,Historical price of memory,Historical price of flash memory,Historical price of disk drives,Historical price of solid-state drives
0,World,OWID_WRL,1957,3.786967e+15,NaN,NaN,NaN
1,World,OWID_WRL,1959,6.032806e+14,NaN,6.747736e+10,NaN
2,World,OWID_WRL,1960,4.588051e+13,NaN,3.150364e+10,NaN
3,World,OWID_WRL,1965,2.170492e+13,NaN,NaN,NaN
4,World,OWID_WRL,1970,4.892936e+12,NaN,1.731185e+09,NaN


In [2]:
df_price = df_price.rename(columns={
    'Historical price of memory': 'Memory',
    'Historical price of flash memory': 'Flash',
    'Historical price of disk drives': 'Disk',
    'Historical price of solid-state drives': 'Solid state'
})

# Sort the original dataframe by Year
df_sorted = df_price.sort_values('Year')

# Then melt the sorted dataframe
df_melted = df_sorted.melt(
    id_vars=['Year'],
    value_vars=['Memory',
                'Flash',
                'Disk',
                'Solid state'],
    var_name='Memory Type',
    value_name='Price'
)

df_melted.head()

,Year,Memory Type,Price
0,1956,Memory,NaN
1,1957,Memory,3.786967e+15
2,1959,Memory,6.032806e+14
3,1960,Memory,4.588051e+13
4,1964,Memory,NaN


In [54]:
import plotly.express as px
import numpy as np

top_title = "#2D2E2D"
sub_title = "#5B5B5B"
y_label_color = "#555555"
explanations_color = "#858585"

color_dict = {
    "Memory": "#5571a1",
    "Flash": "#b33c10",
    "Solid state": "#398b6e",
    "Disk": "#AB865B",
    
}


# ----------------------------------------------------------#
fig = px.line(
    df_melted,
    x="Year",
    y="Price",
    color="Memory Type",
    color_discrete_map=color_dict,
    markers=True,
    log_y=True,
    
)

# Update the layout to avoid connecting gaps
fig.update_traces(connectgaps=True)

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=80,r=120,b=120,t=120),  # give space for text
    plot_bgcolor="white",
    paper_bgcolor="white",
    showlegend=False
)
# ----------------------------------------------------------#

# Define your tick positions (major ticks)
tickvals = [100, 1e4, 1e6, 1e8, 1e10, 1e12, 1e14]

# Define the matching labels
ticktext = [
    '100 $/TB',
    '10,000 $/TB',
    '1 million $/TB',
    '100 million $/TB',
    '10 billion $/TB',
    '1 trillion $/TB',
    '100 trillion $/TB'
]


# ----------------------------------------------------------#
# Update layout with title
fig.update_layout(
    title=dict(
        text=(
            "<span style='font-size:28px; color:top_title; font-weight:bold;'>"
            "Historical price of computer memory and storage</span>"
            "<span style='font-size:16px; color:sub_title;'>"
            '<br><br>This data is expressed in US dollars per terabyte (TB), adjusted for inflation. "Memory" refers to random accces'
            '<br>memory (RAM), "disc" to magnetic storage, "flash" to special memory used for rapid data access and rewriting,'
            '<br>and "solid state" to solid-state drives (SSDs).'
            "</span>"
        ),
        x=0.01,  # Position title
        y=0.95,
        xanchor="left",
    ),
    xaxis_title=None,
    yaxis_title=None,
    xaxis=dict(
        range=[1956, 2023.3],  # Fix the x-axis to desired bounds
        anchor="free",
        position = 0.055,
        tickvals=[1956, 1970, 1980, 1990, 2000, 2010, 2023],  # or auto
        ticktext=["     1956", "1970", "1980", "1990", "2000",  "2010","2023      "],
        tickfont=dict(size=15, color="#555555"),
        ticks="outside",
        showticklabels=True,
        showline=False,   
        linecolor="#ccc"    
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext,
        tickfont=dict(size=15, color=y_label_color),
        showgrid=True,
        gridcolor="#E5E5E5",  # optional: light gray grid lines
        gridwidth=1,
        griddash="dash",
        showline=True,   
        
    )

)

# List of entities in desired vertical order (top to bottom)
entities_ordered = list(color_dict.keys()) 
# Custom vertical positions (same length as entities_ordered)
custom_y_positions = [
    0.18,  # Memory
    0.12, # Flash
    0.08,  # Solid state
    0.05   # Disk
]

for i, entity in enumerate(entities_ordered):
    y_pos = custom_y_positions[i]

    # Draw the annotation
    fig.add_annotation(
        x=1.01,
        y=y_pos,
        xref="paper",
        yref="paper",
        text=entity,
        font=dict(color=color_dict.get(entity, "#000000"), size=15),
        showarrow=False,
        xanchor="left",
        yanchor="middle"
    )


# Update layout with bottom explanation text
fig.add_annotation(
    text=(
        "<span style='font-size:14px; color:" + explanations_color + ";'>"
        "<b>Data source:</b> John C. McCallum (2023); U.S. Bureau of Labor Statistics (2024)"
        "                              OurWorldinData.org/internet | CC BY"
        "<br>"
        "<br><b>Note:</b>For each year, the time series shows the cheapest historical price recorded until that year. This data is expressed in constant 2020"
        "<br>US$."
        "</span>"
    ),
    xref="paper",  # Relative to the plot width (0 to 1)
    yref="paper",  # Relative to the plot height (0 to 1)
    x=-0.18,  # Match title x position (left-aligned)
    y=-0.2,  # Below the plot (adjust as needed)
    showarrow=False,
    align="left",
)

fig.add_layout_image(
    dict(
        source="figures/logo.png",  
        xref="paper", yref="paper",  # Relative to figure, not data
        x=1.1, y=1.11,                  # Bottom-right corner
        sizex=0.08, sizey=0.08,      # Adjust size as needed
        xanchor="right", yanchor="bottom",
        layer="above"               # Draw above plot
    )
)

fig.update_layout(hovermode="x unified")

# Optionally prepare a comment (or use literal string in template)
usd_note = "in constant 2020 US$ per terabyte"


for i, trace in enumerate(fig.data):
    if i == 0:
        trace.hovertemplate = (
            usd_note + "<br>" +
            trace.name + ": %{y:.2f} $/TB<br>" +"<extra></extra>"
        )
    else:
        trace.hovertemplate = (
            trace.name + ": %{y:.2f} $/TB<br><extra></extra>"
        )

fig.show()

fig.write_html("interactive_figures/historical_price.html")

## 2. Price on memory

b) Apply storytelling principles to improve this visualization. You can make several variations if you want to tell different stories.